In [5]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [6]:
#  Import and read the charity_data.csv.
charity_data_df = pd.read_csv("Resources/charity_data.csv")
charity_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
charity_data_df.drop(['EIN'], 1)
charity_data_df

C:\Users\vasan\AppData\Local\Temp/ipykernel_31940/1396709758.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  charity_data_df.drop(['EIN'], 1)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [8]:
# Determine the number of unique values in each column.
charity_data_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
name_counts = charity_data_df['NAME'].value_counts()
name_counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [10]:
# How many name counts > 5
name_counts[name_counts>5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: NAME, Length: 354, dtype: int64

In [11]:
# Choose a cutoff value and create a list of names to be replaced
# use the variable name `names_to_replace
names_to_replace = list(name_counts[name_counts<=5].index)

# Replace the application types in the dataframe with 'Other'
for nm_type in names_to_replace:
    charity_data_df['NAME'] = charity_data_df['NAME'].replace(nm_type, 'Other')

# Check to make sure binning was successful
charity_data_df['NAME'].value_counts()

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64

In [12]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = charity_data_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [13]:
# Choose a cutoff value and create a list of applications to be replaced
# use the variable name `classifications_to_replace`
application_to_replace = list(application_counts[application_counts<1000].index)
  
# Replace in dataframe
for app in application_to_replace:
    charity_data_df['APPLICATION_TYPE'] = charity_data_df['APPLICATION_TYPE'].replace(app,"Other")
    
# Check to make sure binning was successful
charity_data_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [14]:
# look at Classifications value counts for binning
classification_counts = charity_data_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [15]:
# Choose a cutoff value and create a list of classifications to be replaced if counts are less than 1000
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts<1000].index)

# Replace in dataframe
for cl in classifications_to_replace:
    charity_data_df['CLASSIFICATION'] = charity_data_df['CLASSIFICATION'].replace(cl,"Other")

# Check to make sure binning was successful
charity_data_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [16]:
# Generate categorical data list
app_categories = charity_data_df.dtypes[charity_data_df.dtypes == 'object'].index.tolist()
app_categories

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
charity_dummies_df = pd.get_dummies(charity_data_df)
charity_dummies_df.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,10531628,1,108590,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,10547893,1,5000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,10553066,1,6692,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,10556103,1,142590,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [18]:
# Split our preprocessed data into our features and target arrays
X = charity_dummies_df.drop('IS_SUCCESSFUL', axis='columns').values
y = charity_dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# Get the shape of the scaled data
X_train_scaled.shape

(25724, 396)

In [20]:
# Create a neural network model by assigning the number of input features and nodes for each layer using TensorFlow and Keras.
input_features = len(X_train[0])
hidden_layer_1 = 100
hidden_layer_2 = 50
hidden_layer_3 = 20

nn_model = tf.keras.models.Sequential()

# Add a hidden layer with 100 neurons
nn_model.add(tf.keras.layers.Dense(units=hidden_layer_1, input_dim=input_features, activation='relu'))
# Add a hidden layer with 50 neurons
nn_model.add(tf.keras.layers.Dense(units=hidden_layer_2, activation='sigmoid'))
# Add a hidden layer with 20 neurons
nn_model.add(tf.keras.layers.Dense(units=hidden_layer_3, activation='sigmoid'))
# Add an output layer with 1 neuron
nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               39700     
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 20)                1020      
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 45,791
Trainable params: 45,791
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
# Train the model
nn_fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4824 - accuracy: 0.7724
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4368 - accuracy: 0.7948
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4288 - accuracy: 0.7969
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4255 - accuracy: 0.7973
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4230 - accuracy: 0.7979
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4210 - accuracy: 0.7986
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4196 - accuracy: 0.8003
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4177 - accuracy: 0.8005
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4171 - accuracy: 0.8009
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4164 - accura

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4997 - accuracy: 0.7839 - 298ms/epoch - 1ms/step
Loss: 0.4996952414512634, Accuracy: 0.7839066982269287


In [24]:
# Export our model to HDF5 file
nn_model.save('Resources/AlphabetSoupCharity_Optimization.h5')

In [25]:
# Random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [30]:
# Create a random forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# Fit the random forest classifier to the training data
rf_classifier.fit(X_train_scaled, y_train)
# Predict the labels of the test data
rf_predictions = rf_classifier.predict(X_test_scaled)
# Evaluate the accuracy of the model
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Accuracy: {rf_accuracy}")


Accuracy: 0.7439067055393586


In [31]:
# logistic regression classifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [33]:
# Create a logistic regression classifier
log_reg_classifier = LogisticRegression(random_state=42)
# Fit the logistic regression classifier to the training data
log_reg_classifier.fit(X_train_scaled, y_train)
# Predict the labels of the test data
log_reg_predictions = log_reg_classifier.predict(X_test_scaled)
# Evaluate the accuracy of the model
log_reg_accuracy = accuracy_score(y_test, log_reg_predictions)
print(f"Accuracy: {log_reg_accuracy}")

Accuracy: 0.7878717201166181


### Report

1. Explain the purpose of this analysis.<br>
 To select applicants for funding with best chance of success and predict whether applicants will be successful if funded.

2. Using bulleted lists and images to support your answers, address the following questions:

Data Preprocessing<br><br>
    * What variable(s) are the target(s) for your model?<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; The target is the 'IS_SUCCESSFUL' column.<br><br>
    * What variable(s) are the features for your model?<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The features of this model are the NAME, APPLICATION, TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, INCOME_AMT,SPECIAL_CONSIDERATIONS, STATUS, and ASK_AMT columns.<br><br>
    * What variable(s) should be removed from the input data because they are neither targets nor features?<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;We can drop 'STATUS' as it has the same value.

Compiling, Training, and Evaluating the Model<br><br>
    * How many neurons, layers, and activation functions did you select for your neural network model, and why?<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;100,50,20 neurons for the 3 layers were chosen. The first layer was 'relu' and all others 'sigmoid' to increase accuracy of the model.
    <br><br>
    * Were you able to achieve the target model performance?<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
    Yes, I was able to achieve the target model performance by making the above choices.<br><br>
    * What steps did you take in your attempts to increase model performance?<br>
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Choosing 'sigmoid' helped increase performance.<br><br>

Summarize the overall results of the deep learning model. Include a recommendation for how a different model could solve this classification problem, and then explain your recommendation.<br><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;We were able to achieve an increase in accuracy from 72% to 78% by choosing a different classifier and adding more layers with 'sigmoid' as the activation function.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;A model created with Logistic Regression can be recommended as it also has an accuracy of 78% and can solve this classification.
